In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
#Import all the dependencies
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
df = pd.read_csv('lexcial_rich_data.zip')

In [3]:
df.head()

,Polls ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score,time_stamp,tokenized_responses,stemmed_responses,clean_responses,string,LOs/ HCs,...,Summer,Dugast,words_count,unique_words,ttr,rttr,cttr,mtld,herdan,maas
0,12522,The strengths of Plato's approach is his const...,#objectivemorality,2,1,"['The', 'strengths', 'of', ""Plato's"", 'approac...","['the', 'strength', 'of', 'plato', 'approach',...","['strength', 'plato', 'approach', 'construct',...",strength plato approach construct whole framew...,objmorality,...,0.967595,90.603405,51,43,0.843137,6.021204,4.257634,91.035000,0.956604,0.011037
1,12522,In the breakout we discussed if outside the ca...,#objectivemorality,3,1,"['In', 'the', 'breakout', 'we', 'discussed', '...","['in', 'the', 'breakout', 'we', 'discuss', 'if...","['breakout', 'discuss', 'outsid', 'cave', 'big...",breakout discuss outsid cave bigger cave thus ...,objmorality,...,0.946879,56.677811,59,44,0.745763,5.728312,4.050528,64.978667,0.928058,0.017644
2,12522,Back to cmmon confusion time: the section 'und...,#objectivemorality,2,1,"['Back', 'to', 'cmmon', 'confusion', 'time', '...","['back', 'to', 'cmmon', 'confus', 'time', 'the...","['back', 'cmmon', 'confus', 'time', 'section',...",back cmmon confus time section understand inte...,objmorality,...,0.938242,47.879322,43,32,0.744186,4.879954,3.450649,38.068807,0.921444,0.020886
3,12522,Most difficult weakness is that his position w...,#objectivemorality,2,1,"['Most', 'difficult', 'weakness', 'is', 'that'...","['most', 'difficult', 'weak', 'is', 'that', 'h...","['difficult', 'weak', 'posit', 'understand', '...",difficult weak posit understand testabl like i...,objmorality,...,0.964056,80.343345,39,33,0.846154,5.284229,3.736514,70.980000,0.954401,0.012447
4,12522,I'm still trying to understand the significanc...,#objectivemorality,2,1,"[""I'm"", 'still', 'trying', 'to', 'understand',...","[""i'm"", 'still', 'tri', 'to', 'understand', 't...","[""i'm"", 'still', 'tri', 'understand', 'signifi...",i'm still tri understand signific cave analog ...,objmorality,...,0.966813,87.158819,41,35,0.853659,5.466082,3.865103,78.446667,0.957393,0.011473


In [4]:
#only take when the column isn't NaN 
df = df[df['string'].notna()]

In [5]:
from nltk import word_tokenize

df_string = df.string

tagged_data = [TaggedDocument(words=word_tokenize(_d), tags=[str(i)]) for i, _d in enumerate(df_string)]

In [6]:
max_epochs = 10
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

In [7]:
tagged_data[:10]

[TaggedDocument(words=['strength', 'plato', 'approach', 'construct', 'whole', 'framework', 'matt', 'answer', 'preparatori', 'poll', 'focus', 'specif', 'moral', 'knowledg', 'instead', 'set', 'form', 'good', 'framework', 'veri', 'hard', 'attack', 'weak', 'abstract'], tags=['0']),
 TaggedDocument(words=['breakout', 'discuss', 'outsid', 'cave', 'bigger', 'cave', 'thus', 'abl', 'illumin', 'howev', 'still', 'stuck', 'know', 'left', 'cave', 'matt', 'said', 'doesnt', 'matter', 'moral', 'knowledg', 'dont', 'need', 'prove', 'pursu'], tags=['1']),
 TaggedDocument(words=['back', 'cmmon', 'confus', 'time', 'section', 'understand', 'intellig', 'tabl', 'suppos', 'equat', 'knowledg', 'knowledg', 'suppos', 'object', 'true', 'differ', 'moral', 'object', 'moral', 'knowledg', 'question', 'rais', 'rosi', 'follow'], tags=['2']),
 TaggedDocument(words=['difficult', 'weak', 'posit', 'understand', 'testabl', 'like', 'incomprehens', 'peopl', 'struggl', 'quit', 'lot', 'decid', 'whether', 'ani', 'way', 'get', 'ca

In [8]:

for epoch in range(max_epochs):
    # print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=10)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


Note: 
- How to tune a model? 16 min for 10 epoch 

- `model.dv.get_vector(key, norm=True)` key is the tagged key number 

- run get vector from 0 - 10, get the list and append it in the dataset 

In [9]:
vectors_list = pd.Series([model.dv.get_vector(i, norm=True) for i in range(len(df))]).T
len(tagged_data), len(vectors_list)

(176141, 176141)

In [11]:
# import numpy as np
# vectors_list = pd.Series([model.dv.get_vector(i, norm=True) for i in range(len(df))]).T

df.reset_index(drop=True, inplace=True)
df['vec'] = vectors_list
df

,Polls ID,Poll Responses Response,Assessment reports Hashtag,Assessment reports Score,time_stamp,tokenized_responses,stemmed_responses,clean_responses,string,LOs/ HCs,...,Dugast,words_count,unique_words,ttr,rttr,cttr,mtld,herdan,maas,vec
0,12522,The strengths of Plato's approach is his const...,#objectivemorality,2,1,"['The', 'strengths', 'of', ""Plato's"", 'approac...","['the', 'strength', 'of', 'plato', 'approach',...","['strength', 'plato', 'approach', 'construct',...",strength plato approach construct whole framew...,objmorality,...,90.603405,51,43,0.843137,6.021204,4.257634,91.035000,0.956604,0.011037,"[0.2852866, 0.10944329, 0.12894145, 0.04560305..."
1,12522,In the breakout we discussed if outside the ca...,#objectivemorality,3,1,"['In', 'the', 'breakout', 'we', 'discussed', '...","['in', 'the', 'breakout', 'we', 'discuss', 'if...","['breakout', 'discuss', 'outsid', 'cave', 'big...",breakout discuss outsid cave bigger cave thus ...,objmorality,...,56.677811,59,44,0.745763,5.728312,4.050528,64.978667,0.928058,0.017644,"[0.40190294, 0.21657851, -0.058513626, 0.16431..."
2,12522,Back to cmmon confusion time: the section 'und...,#objectivemorality,2,1,"['Back', 'to', 'cmmon', 'confusion', 'time', '...","['back', 'to', 'cmmon', 'confus', 'time', 'the...","['back', 'cmmon', 'confus', 'time', 'section',...",back cmmon confus time section understand inte...,objmorality,...,47.879322,43,32,0.744186,4.879954,3.450649,38.068807,0.921444,0.020886,"[0.36874977, -0.020495353, 0.121122204, -0.052..."
3,12522,Most difficult weakness is that his position w...,#objectivemorality,2,1,"['Most', 'difficult', 'weakness', 'is', 'that'...","['most', 'difficult', 'weak', 'is', 'that', 'h...","['difficult', 'weak', 'posit', 'understand', '...",difficult weak posit understand testabl like i...,objmorality,...,80.343345,39,33,0.846154,5.284229,3.736514,70.980000,0.954401,0.012447,"[0.28095677, 0.32379583, 0.31731528, 0.0348413..."
4,12522,I'm still trying to understand the significanc...,#objectivemorality,2,1,"[""I'm"", 'still', 'trying', 'to', 'understand',...","[""i'm"", 'still', 'tri', 'to', 'understand', 't...","[""i'm"", 'still', 'tri', 'understand', 'signifi...",i'm still tri understand signific cave analog ...,objmorality,...,87.158819,41,35,0.853659,5.466082,3.865103,78.446667,0.957393,0.011473,"[0.19043194, 0.059316073, 0.11514729, 0.262181..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176136,320721,(a) For <s> = each of the energy term is redu...,#IsingModel,4,15312,"['a', 'For', 's', 'each', 'of', 'the', 'energy...","['a', 'for', 's', 'each', 'of', 'the', 'energi...","['energi', 'term', 'reduc', 'partit', 'functio...",energi term reduc partit function entir system...,isingmodel,...,41.516305,100,60,0.600000,6.000000,4.242641,50.852273,0.889076,0.024087,"[0.31073844, 0.45755118, 0.09429593, -0.086379..."
176137,320721,") When <s>!=, then cosh(x) > so function insi...",#IsingModel,4,15312,"['When', 's', 'then', 'cosh', 'x', 'so', 'func...","['when', 's', 'then', 'cosh', 'x', 'so', 'func...","['cosh', 'x', 'function', 'insid', 'log', 'hig...",cosh x function insid log higher magnitud magn...,isingmodel,...,46.999818,60,42,0.700000,5.422177,3.834058,47.659963,0.912886,0.021277,"[0.5535098, 0.014367881, 0.39884728, 0.0066604..."
176138,320721,Emailed it to you earlier. The explanation in ...,#IsingModel,4,15312,"['Emailed', 'it', 'to', 'you', 'earlier', 'The...","['email', 'it', 'to', 'you', 'earlier', 'the',...","['email', 'earlier', 'explan', 'cosh', 'expres...",email earlier explan cosh express larger one t...,isingmodel,...,67.845993,39,32,0.820513,5.124101,3.623287,60.840000,0.946002,0.014739,"[0.37874267, 0.15700613, 0.34322095, 0.1536443..."
176139,320721,. The free energy is F=-NkT*ln(*cosh(dj*beta*<...,#IsingModel,4,15312,"['The', 'free', 'energy', 'is', 'F', 'NkT', 'l...","['the', 'free', 'energi', 'is', 'f', 'nkt', 'l...","['free', 'energi', 'f', 'nkt', 'ln', 'cosh', '...",free energi

In [13]:
vec_df = pd.DataFrame([pd.Series(x) for x in df.vec])
vec_df.columns = ['vec_{}'.format(x+1) for x in vec_df.columns]

vec_df

,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,vec_9,vec_10,vec_11,vec_12,vec_13,vec_14,vec_15,vec_16,vec_17,vec_18,vec_19,vec_20
0,0.285287,0.109443,0.128941,0.045603,-0.073157,-0.508928,-0.077453,0.125095,0.164354,0.147681,0.227378,0.228062,-0.065646,-0.018259,0.589725,0.165162,0.243406,0.041697,0.047487,-0.080301
1,0.401903,0.216579,-0.058514,0.164316,-0.153253,-0.301076,-0.462487,0.327744,0.033768,0.122843,0.067890,-0.172406,-0.026694,-0.313533,0.335412,-0.184949,-0.000822,0.093128,0.022665,0.142223
2,0.368750,-0.020495,0.121122,-0.052415,-0.222401,-0.037515,-0.128381,0.011114,-0.065411,0.389529,0.172736,0.221275,0.215809,-0.060970,0.372512,-0.060175,0.251572,0.366057,0.229787,0.318017
3,0.280957,0.323796,0.317315,0.034841,-0.129726,-0.239208,-0.543972,0.237506,0.026008,-0.006228,0.134589,-0.047511,-0.054336,-0.222005,0.227974,-0.272034,0.009888,-0.173518,0.228723,0.078310
4,0.190432,0.059316,0.115147,0.262182,-0.142157,-0.138226,-0.540430,0.141701,0.132393,0.052895,0.025134,0.092645,-0.150749,-0.375577,-0.011666,-0.353730,0.131935,-0.431555,0.036446,-0.056576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176136,0.310738,0.457551,0.094296,-0.086379,0.198949,0.233082,0.002009,0.091956,-0.338310,0.011298,-0.097236,-0.065367,-0.017717,0.070351,0.154885,0.079737,0.070983,-0.205526,-0.456746,-0.394401
176137,0.553510,0.014368,0.398847,0.006660,0.391471,0.252734,-0.076756,0.109708,-0.220743,0.096808,-0.091754,-0.265362,-0.044340,0.019195,0.226361,0.026845,0.016040,-0.111859,-0.303012,-0.058567
176138,0.378743,0.157006,0.343221,0.153644,0.187442,0.171272,0.170918,0.230688,-0.344162,0.292506,0.147863,0.147491,0.129362,-0.222961,0.042992,-0.094266,0.091049,-0.224582,0.175517,-0.359527
176139,0.378414,0.156319,0.247131,0.134425,0.170150,0.331544,0.077468,0.055609,-0.021488,0.255704,0.096954,-0.268662,-0.076848,-0.322715,-0.006281,0.256984,0.192531,-0.030673,-0.281496,-0.405527


In [17]:
df = pd.concat([df,vec_df], axis = 1)

In [19]:
#covert into a csv
compression_opts = dict(method='zip',
                        archive_name='vectorized_data.csv')  
df.to_csv('vectorized_data.zip', index=False,
          compression=compression_opts)  